In [1]:
import pandas as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

#필요한 라이브러리 가져오기

CSV를 가져오고, MemoryError상의 문제로 데이터 크기를 줄인다. 

In [2]:
credits = pd.read_csv("../credits.csv")
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [6]:
md = pd.read_csv('../movies_metadata.csv')
link_small = pd.read_csv('../links_small.csv')
link_small = link_small[link_small['tmdbId'].notnull()]['tmdbId'].astype('int')
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
movies = md[md['id'].isin(link_small)]

C:\Users\arjec\AppData\Local\Temp\ipykernel_5544\1780651482.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('../movies_metadata.csv')


In [7]:
import json

with open('../keywords.json', encoding='utf-8') as f:
  js = json.loads(f.read())
keywords_df = pd.DataFrame(js)

사용할 컬럼값만 뽑아 데이터프레임으로 만들고, id를 숫자형으로 바꾼 다음 id를 기준으로 하여 하나로 병합한다. 

In [8]:
movies_df = movies[['title', 'id', 'genres']]
credits_df = credits[['cast', 'crew', 'id']]


In [9]:
movies_df.id = pd.to_numeric(movies_df.id, errors='coerce')
credits_df.id = pd.to_numeric(credits_df.id, errors='coerce')
keywords_df.id = pd.to_numeric(keywords_df.id, errors='coerce')

C:\Users\arjec\AppData\Local\Temp\ipykernel_5544\4167628614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.id = pd.to_numeric(movies_df.id, errors='coerce')


In [10]:
movies_df = pd.merge(movies_df, credits_df, on='id', how='inner') #movies와 credits를 먼저 병합했다.
movies_df.head() 

,title,id,genres,cast,crew
0,Toy Story,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,Jumanji,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,Grumpier Old Men,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,Waiting to Exhale,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,Father of the Bride Part II,11862,"[{'id': 35, 'name': 'Comedy'}]","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [11]:
movies_df = pd.merge(movies_df, keywords_df, on='id', how='inner') #그 다음으로 keywords까지 합쳤다.
movies_df.head()

,title,id,genres,cast,crew,keywords
0,Toy Story,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,11862,"[{'id': 35, 'name': 'Comedy'}]","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


배우와 감독 정제하기. 대개 다른 크루들보다도 감독을 많이 보므로 감독을 따로 가져올 것이다. 

In [12]:
from ast import literal_eval

movies_df['cast'] = movies_df['cast'].apply(literal_eval)
movies_df['crew'] = movies_df['crew'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df['cast_size'] = movies_df['cast'].apply((lambda x :len(x)))
movies_df['crew_size'] = movies_df['crew'].apply((lambda x :len(x)))

In [13]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [14]:
movies_df['director'] = movies_df['crew'].apply(get_director)

In [15]:
movies_df['cast'] = movies_df['cast'].apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else []) #배우들도 추출한다. 

In [16]:
movies_df['cast'] = movies_df['cast'].apply(lambda x: x[:3] if len(x) >3 else x) #보통은 주연을 앞에 쓰므로, 가장 영향력이 있을 배우 셋을 빼온다. 

In [17]:
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) #키워드도 함께 가져온다. 

배우와 감독 데이터들을 소문자화시키고, 띄어쓰기도 제거한다. 감독을 강조하기 위해 세 번 적는다. 

In [18]:
movies_df['cast'] = movies_df['cast'].apply(lambda x : [str.lower(i.replace(" ", "")) for i in x])
movies_df['director'] = movies_df['director'].astype(str).apply(lambda x: str.lower(x.replace(" ", "")))
movies_df['director'] = movies_df['director'].apply(lambda x: [x, x, x])

In [19]:
s = movies_df.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keywords'

C:\Users\arjec\AppData\Local\Temp\ipykernel_5544\612500976.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movies_df.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)


In [20]:
s = s.value_counts()

In [21]:
s = s[s>1] #한 영화에밖에 없는 키워드 제거

변형된 단어를 원형으로 바꿔준다. dogs 같은 경우 dog로 통일해 바꿔주는 식이다. 

In [22]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [23]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [24]:
movies_df['keywords'] = movies_df['keywords'].apply(filter_keywords)
movies_df.head()

,title,id,genres,cast,crew,keywords,cast_size,crew_size,director
0,Toy Story,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,"[johnlasseter, johnlasseter, johnlasseter]"
1,Jumanji,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,"[joejohnston, joejohnston, joejohnston]"
2,Grumpier Old Men,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger]",7,4,"[howarddeutch, howarddeutch, howarddeutch]"
3,Waiting to Exhale,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]"
4,Father of the Bride Part II,11862,"[{'id': 35, 'name': 'Comedy'}]","[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",12,7,"[charlesshyer, charlesshyer, charlesshyer]"


In [25]:
movies_df['genres'] = movies_df['genres'].fillna('[]').apply(literal_eval).apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

단어들을 원형으로 만들고 띄어쓰기도 시켜준다. 

In [26]:
movies_df['keywords'] = movies_df['keywords'].apply(lambda x :[stemmer.stem(i) for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [27]:
movies_df['soup'] = movies_df['keywords']+movies_df['cast']+movies_df['director']+movies_df['genres']
movies_df['soup'] = movies_df['soup'].apply(lambda x: ' '.join(x))

In [28]:
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english') #단어별로 벡터화
count_matrix = count.fit_transform(movies_df['soup'])

In [29]:
cosine_sim = cosine_similarity(count_matrix, count_matrix) #코사인 유사도 구하기

인덱스를 영화 이름으로 만든다. 그리고 영화 이름을 입력하면 배우/감독/키워드에 따라 유사한 영화를 추천해주는 함수를 만든다. 

In [30]:
movies_df = movies_df.reset_index()
titles = movies_df['title']
indces = pd.Series(movies_df.index, index=movies_df['title'])

In [37]:
def getrecommandations(title):
    index = indces[title]
    movies_df_scores = list(enumerate(cosine_sim[index]))
    movies_df_scores = sorted(movies_df_scores, key=lambda x:x[1], reverse=True)
    movies_df_scores = movies_df_scores[1:31]
    movies_df_indices = [i[0] for i in movies_df_scores] 
    return titles.iloc[movies_df_indices] 

In [38]:
getrecommandations('The Dark Knight')

8031                 The Dark Knight Rises
6218                         Batman Begins
6623                          The Prestige
2085                             Following
7648                             Inception
4145                              Insomnia
3381                               Memento
8613                          Interstellar
7659            Batman: Under the Red Hood
1134                        Batman Returns
8927               Kidnapping Mr. Heineken
5943                              Thursday
1260                        Batman & Robin
9024    Batman v Superman: Dawn of Justice
4021                  The Long Good Friday
5809                           Point Blank
7362       Gangster's Paradise: Jerusalema
7561                           Harry Brown
7582                              Defendor
8001                      Batman: Year One
2754                          Death Wish 3
132                         Batman Forever
2131                              Superman
2448       